In [1]:
from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core.indices.vector_store.base import VectorStoreIndex
# from llama_index.core.indices.base import BaseIndex

from llama_index.core.base.embeddings.base import BaseEmbedding
from llama_index.core import Settings
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine


class Retriever():
    
    def __init__(self, index: VectorStoreIndex, embed_model: BaseEmbedding, reranker: FlagEmbeddingReranker=None, query_engine: RetrieverQueryEngine=None):
        self.index = index
        self.embed_model = embed_model
        self.reranker = reranker
        self.query_engine = query_engine

    @classmethod
    def load_index_from_disk(cls, PERSIST_DIR, embed_model_name="BAAI/bge-m3", pooling="mean"):
        storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
        embed_model = HuggingFaceEmbedding(
                        model_name=embed_model_name,
                        pooling=pooling # "cls" is default
                    )

        index = load_index_from_storage(storage_context, embed_model=embed_model, llm=None)
        return cls(index=index, embed_model=embed_model)

    def load_reranker(self, top_n=3, reranker_id="Dongjin-kr/ko-reranker"):
        self.reranker = FlagEmbeddingReranker(
                    top_n=top_n,
                    model=reranker_id,
                )

    def load_retriever_query_engine(self, top_k):
        Settings.llm = None
        retriever_query_engine = self.index.as_query_engine(similarity_top_k=top_k, node_postprocessors=[self.reranker], verbose=True)
        self.query_engine = retriever_query_engine
    

    
    def get_k_relevant_documents_with_reranker(self, query):
        relevent_docs = self.query_engine.query(query)
        return relevent_docs        

        
    def get_context_from_relevant_documents(self, relevent_docs):
        context_list = []
        for r in relevent_docs.source_nodes:
            if r.score > 0:
                context_list.append(r.text)
        return context_list


/home/bibekali/anaconda3/envs/hansol-deco-env-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

retriever = Retriever.load_index_from_disk(PERSIST_DIR="../index/train-vector-index-storage-chunk-size-1295", embed_model_name="BAAI/bge-m3")

In [3]:
retriever

In [4]:
retriever.load_reranker(top_n=3, reranker_id="Dongjin-kr/ko-reranker")

In [5]:
retriever.load_retriever_query_engine(top_k=2)

LLM is explicitly disabled. Using MockLLM.


In [6]:
vars(retriever)

{'index': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x73fa8c67f490>,
 'embed_model': HuggingFaceEmbedding(model_name='BAAI/bge-m3', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x73fa89a8ba90>, tokenizer_name='BAAI/bge-m3', max_length=8194, pooling=<Pooling.MEAN: 'mean'>, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None),
 'reranker': FlagEmbeddingReranker(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x73fa89a8ba90>, model='Dongjin-kr/ko-reranker', top_n=3, use_fp16=False),
 'query_engine': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x73fb4032c130>}

In [7]:
relevant_docs = retriever.get_k_relevant_documents_with_reranker(query = "면진장치가 뭐야?")
context_list = retriever.get_context_from_relevant_documents(relevant_docs)
for c in context_list:
    print(c)

TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다., 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다., 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다., 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.


In [34]:
from pydantic import BaseModel

class Augment(BaseModel):
    prompt: str

    @classmethod
    def get_prompt(cls, mode, retriever, question, prev_q=""):
        # prev_q is a must needed for some questions like this: What is the biggest cause of plaster revision? And please tell me how to solve this.”
        if mode=="gpu-solar":
            INSTRUCTION_PROMPT_TEMPLATE = """\
            ### System:
            벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.

            ### User:
            <보고서>
            {CONTEXT}
            </보고서>
            지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
            <질문>
            {QUESTION}
            </질문>

            ### Assistant:
            """
        else:
            INSTRUCTION_PROMPT_TEMPLATE = """\
            <start_of_turn>user
            벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.
            보고서: {CONTEXT}
            질문: {QUESTION}
            <start_of_turn>model
            """

        response_1 = retriever.query_engine.query(question)

        context_list = []
        for r in response_1.source_nodes:
            # print(r.score)
            if r.score > 0:
                if r.score <= 4 and len(context_list) >= 1:
                    pass
                else:
                    context_list.append(r.text)

        # Special case when the follow up question is junk
        if len(context_list) == 0:
            response_2 = retriever.query_engine.query(prev_q + " " + question)
            for r in response_2.source_nodes:
                if r.score > 0:
                    context_list.append(r.text)

        context = prev_q + "\n\n".join(context_list)
        # context = prev_q + "\n\n".join(context_list + [question])

        prompt = INSTRUCTION_PROMPT_TEMPLATE.format(CONTEXT=context, QUESTION=question)   

        return cls(prompt=prompt)     

In [39]:
question = "면진장치가 뭐야?"
augment = Augment.get_prompt("cpu", retriever, question)
print(augment.prompt)

            <start_of_turn>user
            벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.
            보고서: TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다., 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다., 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다., 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.
            질문: 면진장치가 뭐야?
            <start_of_turn>model
            


In [40]:
# import torch
# from transformers import GPT2LMHeadModel
# from transformers import PreTrainedTokenizerFast


# model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
# tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
#   bos_token='</s>', eos_token='</s>', unk_token='<unk>',
#   pad_token='<pad>', mask_token='<mask>')

# text = '근육이 커지기 위해서는'
# text = """
# 벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. '질문'의 의도를 파악하여 정확하게 '보고서'만을 기반으로 답변하세요.
# 보고서: TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다., 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다., 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다., 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.
# 지침사항을 반드시 지키고, '보고서'를 기반으로 '질문'에 답변하세요.
# 질문: 면진장치가 뭐야?
# """
# input_ids = tokenizer.encode(text, return_tensors='pt')
# gen_ids = model.generate(input_ids,
#                          max_length=500,
#                          repetition_penalty=2.0,
#                          pad_token_id=tokenizer.pad_token_id,
#                          eos_token_id=tokenizer.eos_token_id,
#                          bos_token_id=tokenizer.bos_token_id,
#                          use_cache=True)
# generated = tokenizer.decode(gen_ids[0])
# print(generated)


In [11]:
from llama_cpp import Llama

llm = Llama(model_path="/home/bibekali/dacon-hansol-deco-challenge/hansolrag/model_checkpoints/gemma/gemma-2b-it.gguf")

llama_model_loader: loaded meta data with 19 key-value pairs and 164 tensors from /home/bibekali/dacon-hansol-deco-challenge/hansolrag/model_checkpoints/gemma/gemma-2b-it.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-2b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                          gemma.block_count u32              = 18
llama_model_loader: - kv   4:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 8
llama_model_load

In [12]:
type(llm)

llama_cpp.llama.Llama

In [13]:
text = """\
<start_of_turn>user
벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.
보고서: TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다., 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다., 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다., 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.
질문: 면진장치가 뭐야?<end_of_turn>
<start_of_turn>model\
"""

output= llm(text, max_tokens=200)


llama_print_timings:        load time =   51384.35 ms
llama_print_timings:      sample time =      79.03 ms /    42 runs   (    1.88 ms per token,   531.43 tokens per second)
llama_print_timings: prompt eval time =   51382.42 ms /   466 tokens (  110.26 ms per token,     9.07 tokens per second)
llama_print_timings:        eval time =   18838.05 ms /    41 runs   (  459.46 ms per token,     2.18 tokens per second)
llama_print_timings:       total time =   71302.94 ms /   507 tokens


In [14]:
output

{'id': 'cmpl-f829e1f2-747c-4b4d-b736-5be878a93826',
 'object': 'text_completion',
 'created': 1711894400,
 'model': '/home/bibekali/dacon-hansol-deco-challenge/hansolrag/model_checkpoints/gemma/gemma-2b-it.gguf',
 'choices': [{'text': '**면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다.**',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 466, 'completion_tokens': 41, 'total_tokens': 507}}

In [70]:
a = '**면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다.**'
a.lstrip('*').rstrip('*')

'면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다.'

In [80]:
from pydantic import BaseModel
from typing import Any

from peft import AutoPeftModelForCausalLM
import torch
from transformers import BitsAndBytesConfig, AutoTokenizer, pipeline, GPT2LMHeadModel, PreTrainedTokenizerFast
from tqdm import tqdm
import re
from llama_cpp import Llama


class Generator(BaseModel):
    mode: str
    model: Any
    tokenizer: Any


    @classmethod
    def from_config(cls, config_path: str):
        import yaml
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)

        options = config.get('options', [])
        if not options:
            raise ValueError("No options found in the config")
        
        chosen_option = options[0]
        mode = chosen_option['name']

        if chosen_option is None:
            raise ValueError("Chosen mode not found in the config")

        model = cls.get_generative_model(cls, mode, chosen_option['model_id'], chosen_option.get('quantized', False))
        tokenizer = cls.get_tokenizer(cls, mode, chosen_option['model_id'], chosen_option.get('tokenizer_config', {}))

        return cls(
            mode=mode,
            model=model,
            tokenizer=tokenizer
        )
        

    def get_generative_model(self, mode: str, model_id: str, quantized: str):
        if mode == "gpu-solar":
            quantized = quantized.lower() == 'true'
            if quantized:
                bnb_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_use_double_quant=True,
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.half
                )
            else:
                bnb_config = None

            model = AutoPeftModelForCausalLM.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                low_cpu_mem_usage=True,
                quantization_config = bnb_config,
                device_map="auto" #"cuda" if low_cpu_mem_usage=False
            )

            merged_model = model.merge_and_unload()
            return merged_model
        
        elif mode == "cpu-gpt2":
            return GPT2LMHeadModel.from_pretrained(model_id)
    
        elif mode == "cpu-gemini":
            return Llama(model_path=model_id)


    def get_tokenizer(self, mode, model_id, tokenizer_config):
        if mode=="gpu-solar":
            if tokenizer_config.get("eos_token", None) is not None:
                tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, eos_token=tokenizer_config.get("eos_token"))
            else:
                tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
                
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.padding_side = "right"
            return tokenizer
        
        elif mode=="cpu-gpt2":
            tokenizer = PreTrainedTokenizerFast.from_pretrained(model_id, **tokenizer_config)
            return tokenizer
        
        elif mode=="cpu-gemini":
            return None

    
    def generate_text(self, text):
        if self.mode=="gpu-solar":
            pipeline_ = pipeline("text-generation", self.model, tokenizer=self.tokenizer)
            output = pipeline_(text,
                    min_new_tokens=20,
                    max_new_tokens=256,
                    top_p=0.98,
                    top_k=50,
                    temperature=0.9,
                    return_full_text=False,
                    eos_token_id = self.tokenizer.eos_token_id
                    )
            return output[0].get('generated_text').lstrip().rstrip()
        
        elif self.mode=="cpu-gpt2":
            pipeline_ = pipeline("text-generation", self.model, tokenizer=self.tokenizer)
            output = pipeline_(text,
                    min_new_tokens=20,
                    max_new_tokens=256,
                    top_p=0.98,
                    top_k=50,
                    temperature=0.9,
                    return_full_text=False,
                    eos_token_id = self.tokenizer.eos_token_id
                    )
            return output[0].get('generated_text').lstrip().rstrip()
        
        elif self.mode=="cpu-gemini":
            output = self.model(text, max_tokens=256)
            return output.get('choices')[0].get('text').lstrip('*').rstrip('*')
        

    def get_output(self, retriever, query_list):
        responses = []

        for idx, question in tqdm(enumerate(query_list)):
            questions = re.split('[?!.]', question)
            seperate_output = []
            prev_q = ""
            for q in questions:
                if len(q) <= 2:
                    continue
                
                prompt_sample = Augment.get_prompt(self.mode, retriever, q + "?", prev_q=prev_q).prompt
                output = self.generate_text(prompt_sample)
                seperate_output.append(output)
                prev_q = q

                answer = " ".join(seperate_output)
                responses.append(answer)

        return responses


In [72]:
bool('false')



True

In [73]:
generator = Generator.from_config('../config/config.yaml')

llama_model_loader: loaded meta data with 19 key-value pairs and 164 tensors from ../model_checkpoints/gemma/gemma-2b-it.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-2b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                          gemma.block_count u32              = 18
llama_model_loader: - kv   4:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 8
llama_model_loader: - kv   7:              gemma.attention.head_co

In [74]:
vars(generator)

{'mode': 'cpu-gemini',
 'model': <llama_cpp.llama.Llama at 0x73fa3c946a10>,
 'tokenizer': None}

In [75]:
# model = generator.get_generative_model()
# model

In [76]:
# tokenizer = generator.get_tokenizer()
# tokenizer

In [77]:
# tokenizer.eos_token_id

In [78]:
# pipeline = generator.get_text_generation_pipeline(model, tokenizer)

In [83]:
import pandas as pd

a= '/home/bibekali/dacon-hansol-deco-challenge/hansolrag/data/test.csv'
test=pd.read_csv(a)
list(test.질문)




['방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
 '도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?',
 '큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?',
 '철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?',
 '도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?',
 '철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?',
 '어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?',
 '도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 무엇인가요?',
 '도배지에 얼룩이 생기는 다양한 원인들에 대해서 자세히 알려주세요.',
 '고층 건물을 건축할 때 철골구조가 주로 선택되는 이유는 무엇인가요?',
 '유성페인트의 환경 오염에 대한 예시를 알려주세요.',
 '훼손과 오염은 어떻게 다른가요? 또한, 부실 시공으로 인해 타공하자가 발생할 가능성이 있나요?',
 '철골콘크리트 구조에서 사용되는 철근의 재료적 특징은 무엇인가요?"',
 '방염벽지가 실크벽지와 합지벽지와 다른 점은 무엇인가요?',
 '도배지에 곰팡이가 생겼을 때 높은 습도가 원인인 경우, 습기 관리는 어떻게 해야 할까요?"',
 '롱브릭타일은 어떤 종류의 타일인가요? 그리고 페인트 상도재의 역할과 사용 방법에 대해 알려주세요.',
 '준불연재료는 무엇인가요? 그리고 유성페인트를 사용하는 것에 대한 부작용이 있을까요?',
 '어떤 환경 요인이 몰딩 수정을 유발할 수 있는가요? 그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요?',
 '벽에 뚫린 구멍이 이상하다고 하셨는데, 타공 불량이 발생하는 원인과 그 해결 방법에 대해 자세히 설명해주실 수 있을까요?"',
 '다이닝 룸을 더 고급스럽게 꾸밀려면 어떻게 

In [79]:
query_list = ["면진장치가 뭐야?"]
generator.get_output(retriever, query_list)

0it [00:00, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   51384.35 ms
llama_print_timings:      sample time =      65.03 ms /    40 runs   (    1.63 ms per token,   615.14 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   22334.64 ms /    40 runs   (  558.37 ms per token,     1.79 tokens per second)
llama_print_timings:       total time =   23410.13 ms /    41 tokens
1it [00:44, 44.97s/it]


['면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다.']

text = """
벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.
<보고서>
TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다., 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다., 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다., 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.
</보고서>
지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
<질문>
면진장치가 뭐야?
</질문>
"""